## Calculation of the weight matrix

Used Code from Nápoles et al. (2022) article

Github link: https://github.com/LisaKouts/Implicit_bias_FCMs

In [2]:
import math
import numpy as np
import pandas as pd
from scipy.stats import spearmanr, kruskal, pointbiserialr, chi2_contingency, pearsonr, f_oneway, f
from statsmodels.formula.api import ols
from sklearn.preprocessing import OneHotEncoder

# Preprocessing 

1. Load data

In [6]:
df_raw = pd.read_csv('Final_CPS_dataset(30-04-2024_new).csv')

In [14]:
df_raw.sex.value_counts()

sex
1    154427
2    150387
Name: count, dtype: int64

2. Set column names

In [7]:
# Assuming df_raw is your original DataFrame
cols = [
    'age','race', 'sex', 'region_recoded', 'marst', 'nativity', 'classwkr_recoded','annhrs', 'metro', 'wkswork1', 'ft', 'uhrswork', 
    'occupation_category_1990', 'industry_category_1990', 'educ99_lbl_recode', 'bpl_category',
    'incWage_2023_inflated'
]

# Create a new DataFrame with only the specified columns
df_raw = df_raw[cols]

# Sort the new DataFrame by 'incWage_2023_inflated' column
df_raw.sort_values('incWage_2023_inflated', kind='stable', inplace=True, ignore_index=True)

# Display the new DataFrame
df_raw.head()

,age,race,sex,region_recoded,marst,nativity,classwkr_recoded,annhrs,metro,wkswork1,ft,uhrswork,occupation_category_1990,industry_category_1990,educ99_lbl_recode,bpl_category,incWage_2023_inflated
0,35,1,2,5,1,1.0,3,624,3.0,39,0,16,2,12,8,1,4531.951595
1,48,2,2,5,1,1.0,2,624,1.0,52,0,12,3,10,8,1,4531.951595
2,49,1,2,5,1,1.0,2,630,1.0,21,0,30,1,12,9,1,4545.641026
3,33,1,2,6,1,1.0,2,1050,3.0,42,0,25,1,9,12,1,4545.641026
4,27,4,2,5,4,5.0,2,860,1.0,43,0,20,3,12,9,8,4545.641026


In [8]:
columns_to_convert = ['race', 'sex', 'region_recoded', 'marst', 'nativity', 'classwkr_recoded', 'metro', 'ft', 'occupation_category_1990', 'industry_category_1990', 'educ99_lbl_recode', 'bpl_category']

df_raw[columns_to_convert] = df_raw[columns_to_convert].astype('object')
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304814 entries, 0 to 304813
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   age                       304814 non-null  int64  
 1   race                      304814 non-null  object 
 2   sex                       304814 non-null  object 
 3   region_recoded            304814 non-null  object 
 4   marst                     304814 non-null  object 
 5   nativity                  304814 non-null  object 
 6   classwkr_recoded          304814 non-null  object 
 7   annhrs                    304814 non-null  int64  
 8   metro                     304814 non-null  object 
 9   wkswork1                  304814 non-null  int64  
 10  ft                        304814 non-null  object 
 11  uhrswork                  304814 non-null  int64  
 12  occupation_category_1990  304814 non-null  object 
 13  industry_category_1990    304814 non-null  o

In [9]:
nom_cols = df_raw.select_dtypes(include='object').columns
num_cols = df_raw.select_dtypes(exclude='object').columns

3. Recode nominal variables

In [10]:
df_nom = df_raw.loc[:,nom_cols] 
df_nom = df_nom.astype('object')

4. Min-max normalize numeric variables

In [11]:
df_num = df_raw.loc[:,num_cols]
scaled=np.subtract(df_num.values,np.min(df_num.values,axis=0))/np.subtract(np.max(df_num.values,axis=0),np.min(df_num.values,axis=0))
df_num = pd.DataFrame(scaled, columns=df_num.columns)

5. Merge the two

In [12]:
df = pd.concat([df_nom,df_num],axis=1)

In [13]:
df.head()

,race,sex,region_recoded,marst,nativity,classwkr_recoded,metro,ft,occupation_category_1990,industry_category_1990,educ99_lbl_recode,bpl_category,age,annhrs,wkswork1,uhrswork,incWage_2023_inflated
0,1,2,5,1,1.0,3,3.0,0,2,12,8,1,0.256410,0.000000,0.704545,0.045977,0.000000
1,2,2,5,1,1.0,2,1.0,0,3,10,8,1,0.589744,0.000000,1.000000,0.000000,0.000000
2,1,2,5,1,1.0,2,1.0,0,1,12,9,1,0.615385,0.001326,0.295455,0.206897,0.000092
3,1,2,6,1,1.0,2,3.0,0,1,9,12,1,0.205128,0.094164,0.772727,0.149425,0.000092
4,4,2,5,4,5.0,2,1.0,0,3,12,9,8,0.051282,0.052166,0.795455,0.091954,0.000092


# Functions

In [15]:
## Anova, r-squared between continuous protected (DV) and independent nominal
def ols_Rsquared(num_cols, full_dataset, nom_cols):
  """
  Calculate ordinary least squares one-way Anova using statsmodels.formula.api
  ...
  Parameters
  ----------
  num_cols : pandas.index array
    array containing numeric only colummns
    
  full_dataset : pd.DataFrame
    a pd.DataFrame object of containing both numeric and nominal variables

  nom_cols : pandas.index array
    array containing nominal only columns

  Returns
  -------
  pd.DataFrame object with the following columns: (1,2) variables compared with each other, 
  (3) the R-squared, (3) F-statistic, (4) F-critical value, (5) Significant if the
  F-statistic is larger than the F-critical value.
  """
  fits = []
  significance = ''
  for col_num in num_cols:
    for col_nom in nom_cols:
      fit = ols(col_num+' ~ C('+col_nom+')', data = full_dataset).fit()
      k = len(np.unique(df[col_nom]))
      f_critical = f.ppf(q=1-.005, dfn=k-1, dfd=len(full_dataset)-k) # 1-.01 (0.01), 1-.005 (0.05): 95% confidence interval.
      f_stat = fit.fvalue
      if f_stat > f_critical:
        significance = 'significant'
      else:
        significance = 'non-significant'
      fits.append([col_nom,col_num, str(round(fit.rsquared,3)), round(fit.fvalue,3), round(f_critical,2), significance])

  return pd.DataFrame(fits, columns=['Attribute1', 'Attribute2','Correlation', 'F-Statistic', 'F-critical', 'Significance'])

## Pearson R between continuous & continuous
def pearsonR(df_num, full_df):
  """
  Calculate Pearson correlatio coefficient and p-value using 
  scipy.stats.pearson_r
  ...
  Parameters
  ----------
  df_num : pandas.index array
    array containing numeric only colummns
    
  full_df : pd.DataFrame
    a pd.DataFrame object the total set of variables

  Returns
  -------
  pd.DataFrame object with the following columns: (1,2) the variables compared with each other, 
  (2) Pearson's correlation coefficient, (3) the Pi-value, (4) 'F-critical' not applicable here,
  (5) Significant if the Pi_valus is larger than 0.05
  """
  coeff = []
  significance = ''
  for i in df_num:
    for y in df_num:
      rho, pi = pearsonr(full_df[i], full_df[y])
      if pi < 0.05:
        significance = 'significant'
      else:
        significance = 'non-significant'
      coeff.append([i,y,round(rho,3),round(pi,3), 'NaN', significance])
  return pd.DataFrame(coeff, columns=['Attribute1', 'Attribute2','Correlation', 'Pi-value', 'F-critical', 'Significance'])

def cramersv(df_nom, lambda_= None):
  """
  Calculate Cramer's V (multiple categories) coefficient using scipy.stats chi2_contingency
  ...
  Parameters
  ----------
  prot_nominal : pd.DataFrame['column']
    a pd.DataFrame column containing the nominal protected variable
    
  df_nom : pd.DataFrame
    a pd.DataFrame object containing all nominal variables

  lambda_ : pd.DataFrame['column']

  Returns
  -------
  pd.DataFrame object with the following columns: (1,2) the variables compared with each other, 
  (2) Cramer's V coefficient, (3) the Pi-value, (4) 'F-critical' not applicable here,
  (5) Significant if the Pi_valus is larger than 0.05
  """
  p = []
  significance = ''
  for col1 in df_nom:
    for col2 in df_nom:
      if (len(np.unique(df_nom[col1])) == 2) & (len(np.unique(df_nom[col2])) == 2):
        pct=pd.crosstab(df_nom[col1], df_nom[col2]).values
        chi2, pi, _, _=chi2_contingency(pct, lambda_=lambda_)
        corr = np.sqrt(chi2 / len(df_nom)) # Cramer's Phi
        if pi < 0.05:
          significance = 'significant'
        else:
          significance = 'non-significant'
        p.append([col1, col2 ,round(corr,3), round(pi,3), 'NaN', significance])
      else:
        ct=pd.crosstab(df_nom[col1], df_nom[col2]).values
        chi2, pi, _, _=chi2_contingency(ct, lambda_=lambda_)
        corr = np.sqrt(((chi2 / len(df_nom))/min(ct.shape[0]-1,ct.shape[1]-1))) # Cramer's V
        if pi < 0.05:
          significance = 'significant'
        else:
          significance = 'non-significant'
        p.append([col1, col2, round(corr,3), round(pi,3), 'NaN', significance])

  return pd.DataFrame(p, columns=['Attribute1', 'Attribute2', 'Correlation', 'Pi-value', 'F-critical', 'Significance'])

# Weight Matrix

**Pearson's correlation**
1. Continous/Continous: Pearson's r, p-value
2. Continous/nom: OLS, R squared, F-statistic (Pearson's based)
3. Nom/nom (dichotomous & non-dichotomous): Cramer's V (Pearson's based), p-value

In [16]:
df.drop('incWage_2023_inflated', axis=1, inplace=True)

In [17]:
nom_cols = df.select_dtypes(include='object').columns
num_cols = df.select_dtypes(exclude='object').columns

In [18]:
## Secenario 1: continuous continuous
con_con = pearsonR(num_cols, df) # Pi value
## Scenario 2: continuous nominal
con_nom = ols_Rsquared(num_cols, df, nom_cols) # F-statistic
## Scenario 3: nominal nominal
nom_nom = cramersv(df[nom_cols])

In [19]:
# merge the threee scenarios in long format
merge=pd.concat((con_con.iloc[:,[0,1,2,-1]],con_nom.iloc[:,[0,1,2,-1]],nom_nom.iloc[:,[0,1,2,-1]]),ignore_index=True)

# initiate matrix
mat = pd.DataFrame(np.empty((len(df.columns),len(df.columns))),columns=np.sort(df.columns),index=np.sort(df.columns),dtype=str)

# create matrix & add the asterisks to indicate significance
for row_idx in range(len(merge)):
  row=merge.iloc[row_idx,:].values
  with np.printoptions(precision=2, suppress=True):
    cor = str(round(float(row[2]),2))
    if row[-1] == 'significant':
      mat.at[row[0],row[1]] = cor+'*'
      mat.at[row[1],row[0]] = cor+'*'
    else:
      mat.at[row[0],row[1]] = cor
      mat.at[row[1],row[0]] = cor

In [20]:
mat

,age,annhrs,bpl_category,classwkr_recoded,educ99_lbl_recode,ft,industry_category_1990,marst,metro,nativity,occupation_category_1990,race,region_recoded,sex,uhrswork,wkswork1
age,1.0*,0.01*,0.0*,0.01*,0.0*,0.0*,0.01*,0.1*,0.0*,0.0*,0.0*,0.01*,0.0*,0.0*,-0.02*,0.05*
annhrs,0.01*,1.0*,0.0*,0.0*,0.01*,0.31*,0.02*,0.0*,0.0*,0.0*,0.02*,0.0*,0.0*,0.05*,0.88*,0.51*
bpl_category,0.0*,0.0*,1.0*,0.05*,0.19*,0.01*,0.08*,0.06*,0.11*,0.5*,0.11*,0.53*,0.1*,0.07*,0.0*,0.0*
classwkr_recoded,0.01*,0.0*,0.05*,1.0*,0.13*,0.04*,0.28*,0.03*,0.06*,0.06*,0.12*,0.06*,0.04*,0.1*,0.0*,0.0*
educ99_lbl_recode,0.0*,0.01*,0.19*,0.13*,1.0*,0.04*,0.12*,0.06*,0.08*,0.21*,0.25*,0.22*,0.06*,0.1*,0.01*,0.0*
ft,0.0*,0.31*,0.01*,0.04*,0.04*,1.0*,0.2*,0.03*,0.01*,0.01*,0.16*,0.02*,0.05*,0.21*,0.38*,0.01*
industry_category_1990,0.01*,0.02*,0.08*,0.28*,0.12*,0.2*,1.0*,0.05*,0.09*,0.08*,0.46*,0.11*,0.07*,0.4*,0.03*,0.01*
marst,0.1*,0.0*,0.06*,0.03*,0.06*,0.03*,0.05*,1.0*,0.09*,0.06*,0.06*,0.11*,0.04*,0.14*,0.0*,0.0*
metro,0.0*,0.0*,0.11*,0.06*,0.08*,0.01*,0.09*,0.09*,1.0*,0.12*,0.07*,0.16*,0.18*,0.01*,0.0*,0.0*
nativity,0.0*,0.0*,0.5*,0.06*,0.21*,0.01*,0.08*,0.06*,0.12*,1.0*,0.08*,0.35*,0.11*,0.04*,0.0*,0.0


In [21]:
mat= mat.applymap(lambda x: x.replace('*', ''))

C:\Users\Tijl WIjnen\AppData\Local\Temp\ipykernel_25484\2834186992.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mat= mat.applymap(lambda x: x.replace('*', ''))


In [22]:

mat.columns = ['f{}'.format(i) for i in range(1, len(df.columns) + 1)]


### Extracting the values to a CSV file

In [23]:
mat.to_csv('(30-04-2024)_correlation_matrix_CPS_values.csv', index=False)